# Notebook 1: Datenbereitstellung 

**Projekt:** Analyse der Haushaltsausgaben in der Schweiz (2006-2022)  
**Autor:** CAS Information Engineering - ZHAW  
**Version:** 1.0  
**Datum:** 3. Novmeber 2025

---

Dieses Notebook dient als ETL-Schritt und stellt sicher, dass alle Rohdaten sauber und reproduzierbar heruntergeladen werden.
- **1.** Automatisches Abrufen der BFS-Daten per API
- **2.** Bereinigung der Dateinamen
- **3.** Ablage der Originaldateien im Ordner `data/raw`.

In [2]:
import requests
import os
import re
from datetime import datetime

urls = [
    "https://dam-api.bfs.admin.ch/hub/api/dam/assets/32667037",
    "https://dam-api.bfs.admin.ch/hub/api/dam/assets/32667062",
    "https://dam-api.bfs.admin.ch/hub/api/dam/assets/32667068"
]

# Ablage der originalen BFS-Dateien unter data/raw
# Gehe vom notebooks-Ordner ein Verzeichnis hoch, dann in data/raw
download_dir = os.path.join(os.path.dirname(os.getcwd()), "data", "raw")

print(f"📅 Download-Datum: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

os.makedirs(download_dir, exist_ok=True)

def sanitize_filename(filename: str) -> str:
    """Bereinigt den Dateinamen: Umlaute ersetzen, Leerzeichen durch Unterstriche, ungültige Zeichen entfernen."""
    # Umlaute ersetzen
    replacements = {
        "ä": "a",
        "ö": "o",
        "ü": "u",
        "Ä": "A",
        "Ö": "O",
        "Ü": "U"
    }
    for orig, repl in replacements.items():
        filename = filename.replace(orig, repl)
    
    filename = filename.replace(" ", "_") # Leerzeichen zu Unterstriche
    filename = re.sub(r'[<>:"/\\|?*]', '_', filename) # Unzulässige Sonderzeichen entfernen
    filename = filename.replace('\n', '').replace('\r', '') # Entfernen von Steuerzeichen newline & carriage return
    
    return filename.strip()

headers = {"Accept": "application/json"}

for url in urls:
    try:
        # 1️⃣ JSON-Metadaten abrufen
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()

        # 2️⃣ Titel auslesen
        title = data["description"]["titles"]["main"]
        title_clean = sanitize_filename(title)

        # 3️⃣ Master-Dateilink finden
        master_link = None
        file_ext = "dat"
        for link in data.get("links", []):
            if link.get("rel") == "master":
                master_link = link["href"]
                file_ext = link.get("format", file_ext)
                break

        if not master_link:
            print(f"⚠️ Kein master-Link für {url} gefunden.")
            continue

        # 4️⃣ Datei herunterladen
        file_resp = requests.get(master_link)
        file_resp.raise_for_status()

        filename = f"{title_clean}.{file_ext}"
        filepath = os.path.join(download_dir, filename)

        with open(filepath, "wb") as f:
            f.write(file_resp.content)

        print(f"✅ {filename} erfolgreich gespeichert ({len(file_resp.content)} Bytes)")

    except Exception as e:
        print(f"❌ Fehler bei {url}: {e}")


📅 Download-Datum: 2025-10-24 00:15:29
✅ Detaillierte_Haushaltsausgaben_samtlicher_Haushalte_nach_Jahr.xlsx erfolgreich gespeichert (401706 Bytes)
✅ Detaillierte_Haushaltsausgaben_nach_Altersklasse_der_Referenzperson.xlsx erfolgreich gespeichert (918273 Bytes)
✅ Detaillierte_Haushaltsausgaben_nach_Haushaltstyp.xlsx erfolgreich gespeichert (908904 Bytes)
